<a href="https://colab.research.google.com/github/outsider8946/go-algo-hackathon-/blob/main/src/data_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **Предпроцессинг**

In [ ]:
import pandas as pd

In [ ]:
PATH_TO_DATA = '/content/drive/MyDrive/Colab Notebooks/goalgo_hac/data/'
def get_tradestats_by_ticker(ticker:str):
  frames = []
  index = 0
  all_tradestats = ['tradestats_2020.csv','tradestats_2021.csv','tradestats_2022.csv','tradestats_2023.csv']
  for tradestat in all_tradestats:
    print(f'file {tradestat}')
    df = pd.read_csv(PATH_TO_DATA+'tradestats/'+tradestat,sep=';')
    df = df[df['secid']==ticker]
    df = df[df['pr_close']!=df['pr_open']]
    df = df.drop(df[df['tradetime'].str.contains(':05:|:35:')==False].index)
    df = df.drop(['SYSTIME','tradetime','tradedate','secid'],axis=1)
    df.index = range(index,index + len(df))
    frames.append(df)
    index = len(df)+1
  return pd.concat(frames)

In [ ]:
df_sber = get_tradestats_by_ticker('SBER')
print()
df_moex = get_tradestats_by_ticker('MOEX')

file tradestats_2020.csv
file tradestats_2021.csv
file tradestats_2022.csv
file tradestats_2023.csv

file tradestats_2020.csv
file tradestats_2021.csv
file tradestats_2022.csv
file tradestats_2023.csv


In [ ]:
df_sber

,pr_open,pr_high,pr_low,pr_close,pr_std,vol,val,trades,pr_vwap,pr_change,trades_b,trades_s,val_b,val_s,vol_b,vol_s,disb,pr_vwap_b,pr_vwap_s
0,255.97,256.97,255.85,256.77,0.0001,116141.0,297831085.0,1935.0,256.44,0.3125,1091.0,844.0,171039244.0,126791841.0,66691.0,49450.0,0.15,256.47,256.40
1,256.20,256.32,256.09,256.22,0.0001,13225.0,33879268.0,257.0,256.18,0.0078,120.0,137.0,12972945.0,20906323.0,5063.0,8162.0,-0.23,256.23,256.14
2,255.78,256.34,255.52,256.26,0.0001,48369.0,123767265.0,777.0,255.88,0.1877,439.0,338.0,73966037.0,49801228.0,28900.0,19469.0,0.19,255.94,255.80
3,256.64,256.74,256.27,256.50,0.0001,25798.0,66180111.0,440.0,256.53,-0.0546,186.0,254.0,25693661.0,40486449.0,10014.0,15784.0,-0.22,256.58,256.50
4,257.00,257.10,256.77,256.90,0.0001,25299.0,65007573.0,359.0,256.96,-0.0389,222.0,137.0,30354637.0,34652935.0,11813.0,13486.0,-0.07,256.96,256.95
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7771,268.53,268.80,268.50,268.70,0.0000,25773.0,69252793.0,408.0,268.70,0.0633,278.0,130.0,43109719.0,26143073.0,16043.0,9730.0,0.24,268.71,268.69
7772,268.68,268.74,268.51,268.55,0.0000,4894.0,13146346.0,177.0,268.62,-0.0484,87.0,90.0,6758057.0,6388289.0,2516.0,2378.0,0.03,268.60,268.64
7773,268.23,268.24,267.55,267.75,0.0000,31559.0,84526934.0,1012.0,267.84,-0.1790,196.0,816.0,16311124.0,68215810.0,6092.0,25467.0,-0.61,267.75,267.86
7774,268.34,268.44,268.32,268.44,0.0000,33087.0,88792822.0,484.0,268.36,0.0373,394.0,90.0,83221809.0,5571012.0,31011.0,2076.0,0.87,268.36,268.35


# **Заполнение пропусков в данных**

In [ ]:
def fill_and_drop_missings(frame):
  missing_data = frame.isnull()
  total_amount = frame.iloc[:,0].count()
  legal_deviation = total_amount / 2
  list_to_drop = list()

  for column in missing_data.columns.tolist():
    missing_amount = missing_data[column].sum()
    if missing_amount > legal_deviation:
      list_to_drop.append(column)
    else:
      frame[column].fillna(frame[column].mean(), inplace = True)

  if list_to_drop:
    frame = frame.drop(list_to_drop, axis = 1)

  return frame

In [ ]:
df_sber = fill_and_drop_missings(df_sber)

In [ ]:
df_sber.shape

(16714, 19)

In [ ]:
print(df_moex.shape)
df_moex = fill_and_drop_missings(df_moex)
print(df_moex.shape)

(16187, 19)
(16187, 19)


# **Создание таргета**

In [ ]:
def create_target(df:pd.DataFrame):
  df['target'] = 0
  rows = df.shape[0]
  i = 0
  cnt = 0
  temp_target = 0
  while(i!=rows):
    if(cnt%10==0 and cnt !=0):
      cnt = 0
      df['target'].iloc[i] = temp_target
      i-=9
      temp_target = 0
    temp_target+=(df['pr_close'].iloc[i] - df['pr_open'].iloc[i])
    i+=1
    cnt+=1
  return df

In [ ]:
df_sber = create_target(df_sber)
df_moex = create_target(df_moex)

<ipython-input-25-2ba1aee9b2bb>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['target'].iloc[i] = temp_target
<ipython-input-25-2ba1aee9b2bb>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['target'].iloc[i] = temp_target


In [ ]:
df_sber.head(15)

,pr_open,pr_high,pr_low,pr_close,pr_std,vol,val,trades,pr_vwap,pr_change,trades_b,trades_s,val_b,val_s,vol_b,vol_s,disb,pr_vwap_b,pr_vwap_s,target
0,255.97,256.97,255.85,256.77,0.0001,116141.0,297831085.0,1935.0,256.44,0.3125,1091.0,844.0,171039244.0,126791841.0,66691.0,49450.0,0.15,256.47,256.40,0.00
1,256.20,256.32,256.09,256.22,0.0001,13225.0,33879268.0,257.0,256.18,0.0078,120.0,137.0,12972945.0,20906323.0,5063.0,8162.0,-0.23,256.23,256.14,0.00
2,255.78,256.34,255.52,256.26,0.0001,48369.0,123767265.0,777.0,255.88,0.1877,439.0,338.0,73966037.0,49801228.0,28900.0,19469.0,0.19,255.94,255.80,0.00
3,256.64,256.74,256.27,256.50,0.0001,25798.0,66180111.0,440.0,256.53,-0.0546,186.0,254.0,25693661.0,40486449.0,10014.0,15784.0,-0.22,256.58,256.50,0.00
4,257.00,257.10,256.77,256.90,0.0001,25299.0,65007573.0,359.0,256.96,-0.0389,222.0,137.0,30354637.0,34652935.0,11813.0,13486.0,-0.07,256.96,256.95,0.00
5,257.44,257.69,257.32,257.67,0.0001,32875.0,84660814.0,438.0,257.52,0.0893,314.0,124.0,52058838.0,32601975.0,20214.0,12661.0,0.23,257.54,257.50,0.00
6,257.20,257.60,257.20,257.40,0.0001,17967.0,46245685.0,208.0,257.39,0.0778,170.0,38.0,37872173.0,8373512.0,14714.0,3253.0,0.64,257.39,257.41,0.00
7,256.53,256.53,256.32,256.35,0.0001,15590.0,39977774.0,228.0,256.43,-0.0702,84.0,144.0,11034897.0,28942877.0,4303.0,11287.0,-0.45,256.45,256.43,0.00
8,256.45,256.70,256.40,256.58,0.0001,16996.0,43604164.0,215.0,256.56,0.0507,152.0,63.0,35182312.0,8421851.0,13713.0,3283.0,0.61,256.56,256.53,0.00
9,255.40,255.40,255.13,255.20,0.0001,49884.0,127330394.0,565.0,255.25,-0.0783,161.0,404.0,22625417.0,104704976.0,8865.0,41019.0,-0.64,255.22,255.26,0.00


In [ ]:
df_moex

,pr_open,pr_high,pr_low,pr_close,pr_std,vol,val,trades,pr_vwap,pr_change,trades_b,trades_s,val_b,val_s,vol_b,vol_s,disb,pr_vwap_b,pr_vwap_s,target
0,107.88,108.00,107.57,107.74,0.0005,5090.0,5480277.0,224.0,107.67,-0.1298,84.0,140.0,2703730.0,2776546.0,2511.0,2579.0,-0.01,107.68,107.66,0.00
1,108.00,108.02,107.94,107.99,0.0003,1822.0,1967491.0,86.0,107.99,-0.0093,43.0,43.0,725689.0,1241802.0,672.0,1150.0,-0.26,107.99,107.98,0.00
2,108.07,108.19,108.00,108.12,0.0001,6200.0,6701981.0,133.0,108.10,0.0463,92.0,41.0,4695031.0,2006949.0,4343.0,1857.0,0.40,108.11,108.07,0.00
3,108.18,108.20,108.11,108.20,0.0001,1371.0,1482857.0,52.0,108.16,0.0185,37.0,15.0,802607.0,680250.0,742.0,629.0,0.08,108.17,108.15,0.00
4,109.76,109.86,109.67,109.68,0.0002,11600.0,12732643.0,202.0,109.76,-0.0729,143.0,59.0,6916041.0,5816601.0,6299.0,5301.0,0.09,109.80,109.73,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7420,190.43,190.46,190.33,190.44,0.0001,396.0,753946.0,43.0,190.39,0.0053,15.0,28.0,152340.0,601605.0,80.0,316.0,-0.60,190.43,190.38,-0.83
7421,190.54,190.70,190.20,190.24,0.0001,1679.0,3198197.0,113.0,190.48,-0.1574,29.0,84.0,823628.0,2374568.0,432.0,1247.0,-0.49,190.65,190.42,0.05
7422,190.62,190.94,190.58,190.94,0.0001,6364.0,12141688.0,280.0,190.79,0.1679,195.0,85.0,9666702.0,2474986.0,5067.0,1297.0,0.59,190.78,190.82,-0.39
7423,190.89,190.99,190.86,190.92,0.0001,3258.0,6220882.0,179.0,190.94,0.0157,105.0,74.0,3672103.0,2548778.0,1923.0,1335.0,0.18,190.96,190.92,-0.27


# **Сохранение акций отдельной кампании**

In [ ]:
df_sber.to_csv(PATH_TO_DATA + 'sber_tradestats.csv')
df_moex.to_csv(PATH_TO_DATA + 'moex_tradestats.csv')